# CNN Model

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
tf.__version__

'2.17.0'

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_datagen = ImageDataGenerator(
    rescale=1./255,                 # Normalize pixel values to [0, 1]
    rotation_range=10,              # Small rotations to simulate natural variations
    shear_range=0.2,            # Apply shear transformations (within limits)
    width_shift_range=0.1,          # Horizontal translations to augment positional variations
    height_shift_range=0.1,         # Vertical translations to augment positional variations
    zoom_range=0.1,                 # Small zoom to simulate varying distances
    brightness_range=(0.8, 1.2),    # Simulate different lighting conditions
    fill_mode='nearest'             # Fill pixels outside boundaries with nearest pixel value
)
training_set = train_datagen.flow_from_directory('Dataset-Wall-crack-detection',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = train_datagen.flow_from_directory('test_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')


Found 9704 images belonging to 5 classes.
Found 991 images belonging to 5 classes.


In [6]:
# Initializing CNN
cnn = tf.keras.models.Sequential()

# Define the input layer explicitly
cnn.add(tf.keras.layers.Input(shape=(128, 128, 3)))  # Input layer

# Step 1 - Convolution + Pooling
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 2 - Additional Convolution + Pooling Layers
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Additional Convolution + Pooling Layers
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Apply GlobalAveragePooling2D
cnn.add(tf.keras.layers.GlobalAveragePooling2D())

# Fully Connected Layers
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))  # Fully connected layer
cnn.add(tf.keras.layers.Dropout(0.5))  # Dropout for regularization
cnn.add(tf.keras.layers.Dense(units=5, activation='softmax'))  # 5 classes for classification

C:\Users\lenovo\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compiling the CNN
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(x=training_set, validation_data=test_set, epochs=25)  # Increased epochs for better learning


Epoch 1/25


C:\Users\lenovo\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


304/304 ━━━━━━━━━━━━━━━━━━━━ 167s 529ms/step - accuracy: 0.3296 - loss: 1.4314 - val_accuracy: 0.7175 - val_loss: 0.7109
Epoch 2/25
304/304 ━━━━━━━━━━━━━━━━━━━━ 63s 205ms/step - accuracy: 0.6659 - loss: 0.7982 - val_accuracy: 0.7134 - val_loss: 0.7218
Epoch 3/25
304/304 ━━━━━━━━━━━━━━━━━━━━ 59s 193ms/step - accuracy: 0.7007 - loss: 0.7387 - val_accuracy: 0.7608 - val_loss: 0.6236
Epoch 4/25
304/304 ━━━━━━━━━━━━━━━━━━━━ 61s 198ms/step - accuracy: 0.7155 - loss: 0.6864 - val_accuracy: 0.7225 - val_loss: 0.6782
Epoch 5/25
304/304 ━━━━━━━━━━━━━━━━━━━━ 59s 194ms/step - accuracy: 0.7140 - loss: 0.6878 - val_accuracy: 0.7467 - val_loss: 0.6135
Epoch 6/25
304/304 ━━━━━━━━━━━━━━━━━━━━ 61s 198ms/step - accuracy: 0.7301 - loss: 0.6590 - val_accuracy: 0.7750 - val_loss: 0.5673
Epoch 7/25
304/304 ━━━━━━━━━━━━━━━━━━━━ 75s 243ms/step - accuracy: 0.7405 - loss: 0.6345 - val_accuracy: 0.7679 - val_loss: 0.5591
Epoch 8/25
304/304 ━━━━━━━━━━━━━━━━━━━━ 147s 473ms/step - accuracy: 0.7509 - loss: 0.6171 - v

In [8]:
from tensorflow.keras.models import load_model

# Save CNN Model
cnn.save("cnn_crack_detection.h5")

# Load CNN Model
cnn = load_model("cnn_crack_detection.h5")

# VGG16 Model

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
import numpy as np
from tensorflow.keras.preprocessing import image

In [11]:
# DATA PREPROCESSING
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    shear_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=(0.8, 1.2),
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    'Dataset-Wall-crack-detection',
    target_size=(128, 128),  # VGG16 input size
    batch_size=64,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    'test_set',
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'
)

Found 9704 images belonging to 5 classes.
Found 991 images belonging to 5 classes.


In [12]:
# LOAD PRETRAINED VGG16 MODEL (Without Fully Connected Layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False  # Prevent updating weights

# BUILD CUSTOM MODEL
model_vgg16 = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Reduces dimensions
    Dense(128, activation='relu'),
    Dropout(0.5),  # Reduce overfitting
    Dense(5, activation='softmax')  # 8 crack types
])

In [13]:
# Compile the model
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# TRAIN THE MODEL
model_vgg16.fit(training_set, validation_data=test_set, epochs=25)

Epoch 1/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 446s 3s/step - accuracy: 0.4902 - loss: 1.2642 - val_accuracy: 0.7810 - val_loss: 0.6428
Epoch 2/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 442s 3s/step - accuracy: 0.7197 - loss: 0.7080 - val_accuracy: 0.8113 - val_loss: 0.5438
Epoch 3/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 445s 3s/step - accuracy: 0.7434 - loss: 0.6454 - val_accuracy: 0.8224 - val_loss: 0.5034
Epoch 4/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 446s 3s/step - accuracy: 0.7670 - loss: 0.5985 - val_accuracy: 0.8264 - val_loss: 0.4719
Epoch 5/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 448s 3s/step - accuracy: 0.7805 - loss: 0.5663 - val_accuracy: 0.8385 - val_loss: 0.4653
Epoch 6/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 448s 3s/step - accuracy: 0.7800 - loss: 0.5687 - val_accuracy: 0.8295 - val_loss: 0.4616
Epoch 7/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 444s 3s/step - accuracy: 0.7875 - loss: 0.5484 - val_accuracy: 0.8406 - val_loss: 0.4422
Epoch 8/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 445s 3s/step - accuracy: 0.7884 - loss: 0.5391 - val_accu

In [14]:
# Save VGG16 Model
model_vgg16.save("vgg16_crack_detection.h5")  # This "model" is your VGG16 model

# Load VGG16 Model
model_vgg16 = load_model("vgg16_crack_detection.h5")

# ResNet50 Model

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
import numpy as np
from tensorflow.keras.preprocessing import image

In [17]:
# DATA PREPROCESSING
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    shear_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=(0.8, 1.2),
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    'Dataset-Wall-crack-detection',
    target_size=(128, 128),  # ResNet50 input size
    batch_size=64,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    'test_set',
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'
)

Found 9704 images belonging to 5 classes.
Found 991 images belonging to 5 classes.


In [18]:
# LOAD PRETRAINED ResNet50 MODEL (Without Fully Connected Layers)
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the convolutional layers
for layer in base_model_resnet50.layers:
    layer.trainable = False  # Prevent updating weights

# BUILD CUSTOM MODEL
model_resnet50 = Sequential([
    base_model_resnet50,
    GlobalAveragePooling2D(),  # Reduces dimensions efficiently
    Dense(128, activation='relu'),
    Dropout(0.5),  # Reduce overfitting
    Dense(5, activation='softmax')  # 5 crack types
])

In [19]:
# Compile the model
model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# TRAIN THE MODEL
model_resnet50.fit(training_set, validation_data=test_set, epochs=25)

Epoch 1/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 290s 2s/step - accuracy: 0.2677 - loss: 1.6175 - val_accuracy: 0.3935 - val_loss: 1.4581
Epoch 2/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 279s 2s/step - accuracy: 0.4529 - loss: 1.4142 - val_accuracy: 0.6418 - val_loss: 1.2099
Epoch 3/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.5348 - loss: 1.2313 - val_accuracy: 0.6529 - val_loss: 1.0600
Epoch 4/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 276s 2s/step - accuracy: 0.5579 - loss: 1.1225 - val_accuracy: 0.6781 - val_loss: 0.9513
Epoch 5/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.6016 - loss: 1.0475 - val_accuracy: 0.6953 - val_loss: 0.9014
Epoch 6/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 368s 2s/step - accuracy: 0.6145 - loss: 0.9897 - val_accuracy: 0.7003 - val_loss: 0.8571
Epoch 7/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 287s 2s/step - accuracy: 0.6274 - loss: 0.9626 - val_accuracy: 0.6761 - val_loss: 0.8396
Epoch 8/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 276s 2s/step - accuracy: 0.6455 - loss: 0.9125 - val_accu

# MobileNetV2 Model

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

In [22]:
# DATA PREPROCESSING
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    shear_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=(0.8, 1.2),
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    'Dataset-Wall-crack-detection',
    target_size=(128, 128),  # MobileNetV2 input size
    batch_size=64,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    'test_set',
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'
)

Found 9704 images belonging to 5 classes.
Found 991 images belonging to 5 classes.


In [23]:
# LOAD PRETRAINED MobileNetV2 MODEL (Without Fully Connected Layers)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False  # Prevent updating weights

# BUILD CUSTOM MODEL
mobilenet_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Reduces dimensions
    Dense(128, activation='relu'),
    Dropout(0.5),  # Reduce overfitting
    Dense(5, activation='softmax')  # 5 crack types
])

In [24]:
# Compile the model
mobilenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# TRAIN THE MODEL
mobilenet_model.fit(training_set, validation_data=test_set, epochs=25)

# SAVE THE MODEL
mobilenet_model.save("mobilenet_crack_detection.h5")

Epoch 1/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.6037 - loss: 1.0274 - val_accuracy: 0.7477 - val_loss: 0.6030
Epoch 2/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.7606 - loss: 0.6038 - val_accuracy: 0.7941 - val_loss: 0.4935
Epoch 3/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 221s 1s/step - accuracy: 0.7725 - loss: 0.5632 - val_accuracy: 0.8204 - val_loss: 0.4685
Epoch 4/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.7824 - loss: 0.5566 - val_accuracy: 0.8315 - val_loss: 0.4437
Epoch 5/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.7937 - loss: 0.5239 - val_accuracy: 0.8476 - val_loss: 0.4261
Epoch 6/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.7882 - loss: 0.5293 - val_accuracy: 0.8244 - val_loss: 0.4462
Epoch 7/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.7960 - loss: 0.5170 - val_accuracy: 0.8305 - val_loss: 0.4247
Epoch 8/25
152/152 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.7956 - loss: 0.5065 - val_accu

# After evaluating multiple deep learning architectures, including MobileNet, ResNet50, CNN, and VGG16, we found that MobileNet achieved the highest accuracy. Therefore, we have chosen MobileNet as the backbone model for detecting wall cracks, classifying crack types (horizontal, vertical, diagonal, web-shaped), and providing suitable retrofitting techniques based on the identified crack type.

In [54]:
import numpy as np
from keras.preprocessing import image

# Load and preprocess the test image
test_image = image.load_img('single-prediction/123456.jpg', target_size=(128, 128))  # Match CNN input size
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0  # Normalize
test_image = np.expand_dims(test_image, axis=0)

# Predict the crack type
result = mobilenet_model.predict(test_image)
print("Prediction probabilities:", result)

# Mapping of classes to crack types, causes, and retrofitting techniques
class_mapping = {
    0: {
        'type': 'Spiderweb Cracks',
        'cause': 'Poor curing, material defects',
        'retrofit': [
            'Re-plaster the affected area with quality materials.',
            'Use non-shrink grout or fillers to seal gaps.',
            'Conduct proper curing of concrete to prevent recurrence.'
        ]
    },
    1: {
        'type': 'Diagonal Cracks',
        'cause': 'Seismic forces, differential settlement',
        'retrofit': [
            'Apply carbon fiber strips for reinforcement.',
            'Use steel mesh with mortar for re-strengthening.',
            'Re-level the structure if caused by settlement.'
        ]
    },
    2: {
        'type': 'Horizontal Cracks',
        'cause': 'Settlement, soil deformation, thermal stress',
        'retrofit': [
            'Install tie rods or straps to reinforce walls.',
            'Apply epoxy injection to bond cracks.',
            'Add support beams or bracing to reduce stress.'
        ]
    },
    3: {
        'type': 'Non-crack',
        'cause': 'Crack not present',
        'retrofit': ['Retrofitting not required.']
    },
    4: {
        'type': 'Vertical Cracks',
        'cause': 'Shrinkage, uneven foundation, overload',
        'retrofit': [
            'Use polyurethane or epoxy injection for sealing.',
            'Strengthen foundations with underpinning.',
            'Improve soil compaction around the foundation.'
        ]
    },
    5: {
        'type': 'Hairline Cracks',
        'cause': 'Shrinkage, minor settlement',
        'retrofit': [
            'Use surface sealants like acrylic or epoxy.',
            'Monitor cracks for progression over time.',
            'Improve environmental conditions to reduce shrinkage.'
        ]
    },
    6: {
        'type': 'Step Cracks',
        'cause': 'Foundation movement, weak mortar joints',
        'retrofit': [
            'Replace damaged mortar with fresh mortar.',
            'Strengthen masonry with steel reinforcement.',
            'Stabilize the foundation to prevent further movement.'
        ]
    },
    7: {
        'type': 'Wide/Deep Cracks',
        'cause': 'Structural instability, heavy loads',
        'retrofit': [
            'Install steel or concrete anchors to stabilize the structure.',
            'Use epoxy injection for sealing.',
            'Reinforce with additional steel rods or plates.'
        ]
    }
}

# Get the predicted class index
predicted_class = np.argmax(result)

# Handle predictions and display retrofitting techniques
if predicted_class in class_mapping:
    prediction = class_mapping[predicted_class]
    print(f"Predicted Crack Type: {prediction['type']}")
    print(f"Likely Cause of Crack: {prediction['cause']}")
    print("Retrofitting Techniques:")
    for technique in prediction['retrofit']:
        print(f"- {technique}")
else:
    print("Prediction confidence too low or unknown class!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Prediction probabilities: [[8.3931220e-01 1.0083478e-01 5.9287813e-02 4.2981910e-07 5.6485197e-04]]
Predicted Crack Type: Spiderweb Cracks
Likely Cause of Crack: Poor curing, material defects
Retrofitting Techniques:
- Re-plaster the affected area with quality materials.
- Use non-shrink grout or fillers to seal gaps.
- Conduct proper curing of concrete to prevent recurrence.


In [56]:
import pickle
filename = 'training_set.sav'
pickle.dump(cnn, open(filename, 'wb'))

In [27]:
# loading the saved model
loaded_model = pickle.load(open('training_set.sav', 'rb'))

C:\Users\lenovo\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [28]:
import numpy as np
from keras.preprocessing import image

# Load and preprocess the test image
test_image = image.load_img('single-prediction/123456.jpg', target_size=(128, 128))  # Match CNN input size
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0  # Normalize
test_image = np.expand_dims(test_image, axis=0)

# Predict the crack type
result = loaded_model.predict(test_image)
print("Prediction probabilities:", result)

# Mapping of classes to crack types, causes, and retrofitting techniques
class_mapping = {
    0: {
        'type': 'Spiderweb Cracks',
        'cause': 'Poor curing, material defects',
        'retrofit': [
            'Re-plaster the affected area with quality materials.',
            'Use non-shrink grout or fillers to seal gaps.',
            'Conduct proper curing of concrete to prevent recurrence.'
        ]
    },
    1: {
        'type': 'Diagonal Cracks',
        'cause': 'Seismic forces, differential settlement',
        'retrofit': [
            'Apply carbon fiber strips for reinforcement.',
            'Use steel mesh with mortar for re-strengthening.',
            'Re-level the structure if caused by settlement.'
        ]
    },
    2: {
        'type': 'Horizontal Cracks',
        'cause': 'Settlement, soil deformation, thermal stress',
        'retrofit': [
            'Install tie rods or straps to reinforce walls.',
            'Apply epoxy injection to bond cracks.',
            'Add support beams or bracing to reduce stress.'
        ]
    },
    3: {
        'type': 'Non-crack',
        'cause': 'Crack not present',
        'retrofit': ['Retrofitting not required.']
    },
    4: {
        'type': 'Vertical Cracks',
        'cause': 'Shrinkage, uneven foundation, overload',
        'retrofit': [
            'Use polyurethane or epoxy injection for sealing.',
            'Strengthen foundations with underpinning.',
            'Improve soil compaction around the foundation.'
        ]
    },
    5: {
        'type': 'Hairline Cracks',
        'cause': 'Shrinkage, minor settlement',
        'retrofit': [
            'Use surface sealants like acrylic or epoxy.',
            'Monitor cracks for progression over time.',
            'Improve environmental conditions to reduce shrinkage.'
        ]
    },
    6: {
        'type': 'Step Cracks',
        'cause': 'Foundation movement, weak mortar joints',
        'retrofit': [
            'Replace damaged mortar with fresh mortar.',
            'Strengthen masonry with steel reinforcement.',
            'Stabilize the foundation to prevent further movement.'
        ]
    },
    7: {
        'type': 'Wide/Deep Cracks',
        'cause': 'Structural instability, heavy loads',
        'retrofit': [
            'Install steel or concrete anchors to stabilize the structure.',
            'Use epoxy injection for sealing.',
            'Reinforce with additional steel rods or plates.'
        ]
    }
}

# Get the predicted class index
predicted_class = np.argmax(result)

# Handle predictions and display retrofitting techniques
if predicted_class in class_mapping:
    prediction = class_mapping[predicted_class]
    print(f"Predicted Crack Type: {prediction['type']}")
    print(f"Likely Cause of Crack: {prediction['cause']}")
    print("Retrofitting Techniques:")
    for technique in prediction['retrofit']:
        print(f"- {technique}")
else:
    print("Prediction confidence too low or unknown class!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
Prediction probabilities: [[9.9223000e-01 4.0314151e-03 3.7379717e-03 2.5335298e-08 5.7651545e-07]]
Predicted Crack Type: Spiderweb Cracks
Likely Cause of Crack: Poor curing, material defects
Retrofitting Techniques:
- Re-plaster the affected area with quality materials.
- Use non-shrink grout or fillers to seal gaps.
- Conduct proper curing of concrete to prevent recurrence.
